In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import json
import os
import threading

# Initialisation du driver Selenium
driver = webdriver.Chrome()  # Remplace par le driver de ton choix (Chrome, Firefox, etc.)

def close_all_popups():
    """
    Ferme tous les pop-ups, notifications et publicités détectés.
    """
    popup_selectors = [
        'div[class*="popup"]',  # Pop-ups génériques
        'div[class*="modal"]',  # Modales
        'div[class*="advertisement"]',  # Publicités
        'div[class*="notification"]',  # Notifications
        'button[class*="close"]',  # Boutons de fermeture
        'button[class*="dismiss"]',  # Boutons de rejet
        'button[class*="decline"]',  # Boutons de refus (ex: cookies)
        'img[class*="ad"]',  # Images publicitaires
        'iframe[src*="ads"]',  # Iframes de publicités
        'div[id*="cookie"]',  # Bannières de cookies
        'div[id*="consent"]',  # Bannières de consentement
        'div[id*="overlay"]',  # Overlays
        'div[id*="popup"]',  # Pop-ups par ID
    ]

    for selector in popup_selectors:
        try:
            elements = driver.find_elements(By.CSS_SELECTOR, selector)
            for element in elements:
                try:
                    if "close" in selector.lower():
                        element.click()
                    else:
                        driver.execute_script("arguments[0].style.display = 'none';", element)
                    print(f"Élément fermé ou masqué avec le sélecteur : {selector}")
                except Exception as e:
                    print(f"Erreur lors de la fermeture de l'élément {selector} : {e}")
        except Exception as e:
            print(f"Erreur lors de la recherche de l'élément {selector} : {e}")

def check_for_popups_periodically(interval=2):
    """
    Vérifie périodiquement la présence de pop-ups et les ferme.
    :param interval: Intervalle de vérification en secondes
    """
    try:
        while True:
            close_all_popups()
            time.sleep(interval)
    except Exception as e:
        print(f"Erreur lors de la vérification périodique des pop-ups : {e}")

def click_sur_plus():
    """
    Charge la page et clique sur le bouton "Afficher plus" pour charger plus d'articles.
    """
    driver.get('https://fr.le360.ma/societe/')
    close_all_popups()

    try:
        previous_article_count = 0
        for _ in range(500):
            close_all_popups()
            bouton_afficher_plus = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '.results-list-load-more'))
            )
            bouton_afficher_plus.click()
            time.sleep(2)

            # Vérifier si de nouveaux articles ont été chargés
            current_article_count = len(get_article_links(driver))  # Appel corrigé
            if current_article_count == previous_article_count:
                print("Aucun nouvel article chargé, arrêt du clic sur 'Afficher plus'.")
                break
            previous_article_count = current_article_count

    except Exception as e:
        print(f"Erreur lors du clic sur 'Afficher plus' : {e}")
    finally:
        print("Chargement des articles terminé.")

def get_article_links(driver, max_articles=1500):
    """
    Récupère les liens des articles.
    :param driver: Instance du navigateur Selenium
    :param max_articles: Nombre maximum d'articles à récupérer
    :return: Liste des liens des articles
    """
    for _ in range(2):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = soup.find_all('a', href=True)
    article_links = set()

    for article in articles:
        if len(article_links) >= max_articles:  # Ici, max_articles doit être un entier
            break
        link = article['href']
        if link.startswith('/societe/'):  # Filtre les liens de la catégorie "culture"
            article_links.add(link)

    return list(article_links)

def scrape_article(driver, article_url):
    """
    Scrape les détails d'un article à partir de son URL.
    :param driver: Instance du navigateur Selenium
    :param article_url: URL de l'article à scraper
    :return: Dictionnaire contenant le titre, la date, la catégorie et le contenu de l'article
    """
    try:
        driver.execute_script(f"window.open('{article_url}', '_blank');")
        driver.switch_to.window(driver.window_handles[1])
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        close_all_popups()

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        titre = extract_titre(soup)
        date = extract_date(soup)
        categorie = extract_categorie(soup)
        contenu = extract_contenu(soup)

    except Exception as e:
        print(f"Erreur lors du scraping de l'article {article_url}: {e}")
        titre, date, categorie, contenu = "Titre non trouvé", "Date non trouvée", "Catégorie non trouvée", "Contenu non trouvé"

    finally:
        try:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except Exception as e:
            print(f"Erreur lors de la fermeture de l'onglet : {e}")

    return {
        'titre': titre,
        'date': date,
        'categorie': categorie,
        'contenu': contenu,
    }

def extract_titre(soup):
    """Extrait le titre de l'article."""
    try:
        return soup.find('h1').text.strip()
    except AttributeError:
        return "Titre non trouvé"

def extract_date(soup):
    """Extrait la date de l'article."""
    try:
        return soup.find('div', {"class": "subheadline-date"}).text.strip()
    except AttributeError:
        return "Date non trouvée"

def extract_categorie(soup):
    """Extrait la catégorie de l'article."""
    try:
        return soup.find('a', {"class": "overline-link"}).text.strip()
    except AttributeError:
        return "Catégorie non trouvée"

def extract_contenu(soup):
    """Extrait le contenu de l'article."""
    try:
        paragraphes = soup.find_all('p')
        return " ".join([p.text.strip() for p in paragraphes])
    except AttributeError:
        return "Contenu non trouvé"

def save_to_json(data, filename="articles.json"):
    """
    Sauvegarde les données dans un fichier JSON.
    :param data: Données à sauvegarder
    :param filename: Nom du fichier JSON
    """
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"Données sauvegardées dans {filename}")

def load_scraped_urls(filename="le360.json"):
    """
    Charge les URLs déjà scrapés depuis un fichier JSON.
    :param filename: Nom du fichier JSON
    :return: Un ensemble des URLs déjà scrapés
    """
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            data = json.load(f)
            return set(article['url'] for article in data)
    return set()

if __name__ == "__main__":
    try:
        popup_thread = threading.Thread(target=check_for_popups_periodically, daemon=True)
        popup_thread.start()

        click_sur_plus()

        article_urls = get_article_links(driver, max_articles=1200)
        print(f"{len(article_urls)} articles trouvés.")

        scraped_urls = load_scraped_urls()

        articles_data = []
        for url in article_urls:
            if url in scraped_urls:
                print(f"Article déjà scrapé : {url}")
                continue

            print(f"Scraping de l'article : {url}")
            article_data = scrape_article(driver, url)
            article_data['url'] = url
            articles_data.append(article_data)
            scraped_urls.add(url)

        save_to_json(articles_data)

    except Exception as e:
        print(f"Erreur lors de l'exécution du programme : {e}")
    finally:
        driver.quit()

Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Erreur lors du clic sur 'Afficher plus' : Message: element click intercepted: Element <button class="results-list-load-more">...</button> is not clickable at point (387, 19). Other element would receive the click: <div class="header-360_nav-wrapper-sections header-360_nav-wrapper-fixed desktop" style="background-color: rgb(137,

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import json
import os

# Initialisation du driver Selenium
driver = webdriver.Chrome()  # Remplace par le driver de ton choix (Chrome, Firefox, etc.)

def close_all_popups():
    """
    Ferme tous les pop-ups, notifications et publicités détectés.
    """
    popup_selectors = [
        'div[class*="popup"]',  # Pop-ups génériques
        'div[class*="modal"]',  # Modales
        'div[class*="advertisement"]',  # Publicités
        'div[class*="notification"]',  # Notifications
        'button[class*="close"]',  # Boutons de fermeture
        'img[class*="ad"]',  # Images publicitaires
        'iframe[src*="ads"]',  # Iframes de publicités
    ]

    for selector in popup_selectors:
        try:
            # Fermer les éléments détectés
            elements = driver.find_elements(By.CSS_SELECTOR, selector)
            for element in elements:
                try:
                    # Si c'est un bouton de fermeture, cliquer dessus
                    if "close" in selector.lower():
                        element.click()
                    else:
                        # Sinon, masquer ou supprimer l'élément
                        driver.execute_script("arguments[0].style.display = 'none';", element)
                    print(f"Élément fermé ou masqué avec le sélecteur : {selector}")
                except Exception as e:
                    print(f"Erreur lors de la fermeture de l'élément {selector} : {e}")
        except Exception as e:
            print(f"Erreur lors de la recherche de l'élément {selector} : {e}")

def check_for_popups_periodically(interval=2):
    """
    Vérifie périodiquement la présence de pop-ups et les ferme.
    :param interval: Intervalle de vérification en secondes
    """
    try:
        while True:
            close_all_popups()  # Fermer tous les pop-ups détectés
            time.sleep(interval)  # Attendre avant la prochaine vérification
    except Exception as e:
        print(f"Erreur lors de la vérification périodique des pop-ups : {e}")

def click_sur_plus(lien):
    """
    Charge la page et clique sur le bouton "Afficher plus" pour charger plus d'articles.
    """
    # Charger la page
    driver.get(lien)

    # Fermer tous les pop-ups initiaux
    close_all_popups()

    # Cliquer sur le bouton "Afficher plus"
    try:
        for _ in range(5):  # Limite de 15 clics (ajuste selon tes besoins)
            # Fermer les pop-ups avant chaque clic
            close_all_popups()

            bouton_afficher_plus = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '.results-list-load-more'))
            )
            bouton_afficher_plus.click()
            time.sleep(2)  # Attendre que la page se charge
    except Exception as e:
        print(f"Erreur lors du clic sur 'Afficher plus' : {e}")
    finally:
        print("Chargement des articles terminé.")

def get_article_links(driver, categorie, max_articles=3):
    """
    Récupère les liens des articles.
    :param driver: Instance du navigateur Selenium
    :param categorie: Catégorie des articles à filtrer
    :param max_articles: Nombre maximum d'articles à récupérer
    :return: Liste des liens des articles
    """
    # Faire défiler la page pour charger plus d'articles
    for _ in range(2):  # Défiler 2 fois (ajuste selon le besoin)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    # Parser le HTML de la page
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Trouver les liens des articles
    articles = soup.find_all('a', href=True)
    article_links = set()  # Utiliser un ensemble pour éviter les doublons

    for article in articles:
        if len(article_links) >= max_articles:
            break
        link = article['href']
        if link.startswith(categorie):  # Filtrer les liens des articles
            article_links.add(link)

    return list(article_links)

def scrape_article(driver, article_url):
    """
    Scrape les détails d'un article à partir de son URL.
    :param driver: Instance du navigateur Selenium
    :param article_url: URL de l'article à scraper
    :return: Dictionnaire contenant le titre, la date, la catégorie et le contenu de l'article
    """
    try:
        # Ouvrir l'article dans un nouvel onglet
        driver.execute_script(f"window.open('{article_url}', '_blank');")
        driver.switch_to.window(driver.window_handles[1])  # Basculer vers le nouvel onglet

        # Attendre que la page se charge complètement
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, 'body')))
        
        # Fermer les pop-ups sur la page de l'article
        close_all_popups()

        # Parser le HTML de la page de l'article
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Extraire les données de l'article
        titre = extract_titre(soup)
        date = extract_date(soup)
        categorie = extract_categorie(soup)
        contenu = extract_contenu(soup)

    except Exception as e:
        print(f"Erreur lors du scraping de l'article {article_url}: {e}")
        titre, date, categorie, contenu = "Titre non trouvé", "Date non trouvée", "Catégorie non trouvée", "Contenu non trouvé"

    finally:
        # Fermer l'onglet de l'article et revenir à l'onglet principal
        try:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except Exception as e:
            print(f"Erreur lors de la fermeture de l'onglet : {e}")

    return {
        'titre': titre,
        'date': date,
        'categorie': categorie,
        'contenu': contenu,
    }

def extract_titre(soup):
    """Extrait le titre de l'article."""
    try:
        return soup.find('h1').text.strip()
    except AttributeError:
        return "Titre non trouvé"

def extract_date(soup):
    """Extrait la date de l'article."""
    try:
        return soup.find('div', {"class": "subheadline-date"}).text.strip()
    except AttributeError:
        return "Date non trouvée"

def extract_categorie(soup):
    """Extrait la catégorie de l'article."""
    try:
        return soup.find('a', {"class": "overline-link"}).text.strip()
    except AttributeError:
        return "Catégorie non trouvée"

def extract_contenu(soup):
    """Extrait le contenu de l'article."""
    try:
        paragraphes = soup.find_all('p')
        return " ".join([p.text.strip() for p in paragraphes])
    except AttributeError:
        return "Contenu non trouvé"

def save_to_json(data, filename="articles.json"):
    """
    Sauvegarde les données dans un fichier JSON.
    :param data: Données à sauvegarder
    :param filename: Nom du fichier JSON
    """
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"Données sauvegardées dans {filename}")

def load_scraped_urls(filename="articles.json"):
    """
    Charge les URLs déjà scrapés depuis un fichier JSON.
    :param filename: Nom du fichier JSON
    :return: Un ensemble des URLs déjà scrapés
    """
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            data = json.load(f)
            return set(article['url'] for article in data)
    return set()
    

In [6]:
# CODE AMELIORER


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import json
import os
import threading


def initialize_driver():
    """
    Initialise le pilote Selenium.
    """
    try:
        driver = webdriver.Chrome()  # Remplace par le driver de ton choix (Chrome, Firefox, etc.)
        return driver
    except Exception as e:
        print(f"Erreur lors de l'initialisation du pilote Selenium : {e}")
        return None

def close_all_popups(driver):
    """
    Ferme tous les pop-ups, notifications et publicités détectés.
    """
    popup_selectors = [
        'div[class*="popup"]',  # Pop-ups génériques
        'div[class*="modal"]',  # Modales
        'div[class*="advertisement"]',  # Publicités
        'div[class*="notification"]',  # Notifications
        'button[class*="close"]',  # Boutons de fermeture
        'button[class*="dismiss"]',  # Boutons de rejet
        'button[class*="decline"]',  # Boutons de refus (ex: cookies)
        'img[class*="ad"]',  # Images publicitaires
        'iframe[src*="ads"]',  # Iframes de publicités
        'div[id*="cookie"]',  # Bannières de cookies
        'div[id*="consent"]',  # Bannières de consentement
        'div[id*="overlay"]',  # Overlays
        'div[id*="popup"]',  # Pop-ups par ID
    ]

    for selector in popup_selectors:
        try:
            elements = driver.find_elements(By.CSS_SELECTOR, selector)
            for element in elements:
                try:
                    if "close" in selector.lower():
                        element.click()
                    else:
                        driver.execute_script("arguments[0].style.display = 'none';", element)
                    print(f"Élément fermé ou masqué avec le sélecteur : {selector}")
                except Exception as e:
                    print(f"Erreur lors de la fermeture de l'élément {selector} : {e}")
        except (MaxRetryError, NewConnectionError) as e:
            print(f"Erreur de connexion lors de la recherche de l'élément {selector} : {e}")
            # Redémarrer le pilote Selenium
            driver.quit()
            driver = initialize_driver()
            if driver:
                close_all_popups(driver)
        except Exception as e:
            print(f"Erreur lors de la recherche de l'élément {selector} : {e}")

def check_for_popups_periodically(interval=2):
    """
    Vérifie périodiquement la présence de pop-ups et les ferme.
    :param interval: Intervalle de vérification en secondes
    """
    try:
        while True:
            close_all_popups()
            time.sleep(interval)
    except Exception as e:
        print(f"Erreur lors de la vérification périodique des pop-ups : {e}")


def click_sur_plus():
    """
    Charge la page et clique sur le bouton "Afficher plus" pour charger plus d'articles.
    """
    driver.get('https://fr.le360.ma/societe/')
    close_all_popups()

    try:
        previous_article_count = 0
        for _ in range(500):
            close_all_popups()

            # Faire défiler la page pour dégager le bouton
            driver.execute_script("window.scrollBy(0, 100);")

            # Attendre que le bouton soit cliquable
            bouton_afficher_plus = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '.results-list-load-more'))
            )

            # Cliquer via JavaScript pour éviter l'interception
            driver.execute_script("arguments[0].click();", bouton_afficher_plus)

            time.sleep(2)

            # Vérifier si de nouveaux articles ont été chargés
            current_article_count = len(get_article_links(driver))
            if current_article_count == previous_article_count:
                print("Aucun nouvel article chargé, arrêt du clic sur 'Afficher plus'.")
                break
            previous_article_count = current_article_count

    except Exception as e:
        print(f"Erreur lors du clic sur 'Afficher plus' : {e}")
    finally:
        print("Chargement des articles terminé.")

def get_article_links(driver, max_articles=1500):
    """
    Récupère les liens des articles.
    :param driver: Instance du navigateur Selenium
    :param max_articles: Nombre maximum d'articles à récupérer
    :return: Liste des liens des articles
    """
    for _ in range(2):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = soup.find_all('a', href=True)
    article_links = set()

    for article in articles:
        if len(article_links) >= max_articles:  # Ici, max_articles doit être un entier
            break
        link = article['href']
        if link.startswith('/societe/'):  # Filtre les liens de la catégorie "culture"
            article_links.add(link)

    return list(article_links)

def scrape_article(driver, article_url):
    """
    Scrape les détails d'un article à partir de son URL.
    :param driver: Instance du navigateur Selenium
    :param article_url: URL de l'article à scraper
    :return: Dictionnaire contenant le titre, la date, la catégorie et le contenu de l'article
    """
    try:
        driver.execute_script(f"window.open('{article_url}', '_blank');")
        driver.switch_to.window(driver.window_handles[1])
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        close_all_popups()

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        titre = extract_titre(soup)
        date = extract_date(soup)
        categorie = extract_categorie(soup)
        contenu = extract_contenu(soup)

    except Exception as e:
        print(f"Erreur lors du scraping de l'article {article_url}: {e}")
        titre, date, categorie, contenu = "Titre non trouvé", "Date non trouvée", "Catégorie non trouvée", "Contenu non trouvé"

    finally:
        try:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except Exception as e:
            print(f"Erreur lors de la fermeture de l'onglet : {e}")

    return {
        'titre': titre,
        'date': date,
        'categorie': categorie,
        'contenu': contenu,
    }

def extract_titre(soup):
    """Extrait le titre de l'article."""
    try:
        return soup.find('h1').text.strip()
    except AttributeError:
        return "Titre non trouvé"

def extract_date(soup):
    """Extrait la date de l'article."""
    try:
        return soup.find('div', {"class": "subheadline-date"}).text.strip()
    except AttributeError:
        return "Date non trouvée"

def extract_categorie(soup):
    """Extrait la catégorie de l'article."""
    try:
        return soup.find('a', {"class": "overline-link"}).text.strip()
    except AttributeError:
        return "Catégorie non trouvée"

def extract_contenu(soup):
    """Extrait le contenu de l'article."""
    try:
        paragraphes = soup.find_all('p')
        return " ".join([p.text.strip() for p in paragraphes])
    except AttributeError:
        return "Contenu non trouvé"

def load_existing_data(filename="articles.json"):
    """
    Charge les données existantes depuis un fichier JSON.
    :param filename: Nom du fichier JSON
    :return: Liste des articles existants
    """
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            return json.load(f)
    return []

def save_to_json(data, filename="articles.json"):
    """
    Sauvegarde les données dans un fichier JSON.
    :param data: Données à sauvegarder
    :param filename: Nom du fichier JSON
    """
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"Données sauvegardées dans {filename}")

def load_scraped_urls(filename="le360.json"):
    """
    Charge les URLs déjà scrapés depuis un fichier JSON.
    :param filename: Nom du fichier JSON
    :return: Un ensemble des URLs déjà scrapés
    """
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            data = json.load(f)
            return set(article['url'] for article in data)
    return set()

if __name__ == "__main__":
    try:
        popup_thread = threading.Thread(target=check_for_popups_periodically, daemon=True)
        popup_thread.start()

        click_sur_plus()

        article_urls = get_article_links(driver, max_articles=1200)
        print(f"{len(article_urls)} articles trouvés.")

        scraped_urls = load_scraped_urls()

        existing_data = load_existing_data()
        existing_urls = set(article['url'] for article in existing_data)

        articles_data = []
        for url in article_urls:
            if url in existing_urls:
                print(f"Article déjà présent : {url}")
                continue

            print(f"Scraping de l'article : {url}")
            article_data = scrape_article(driver, url)
            article_data['url'] = url
            articles_data.append(article_data)
            existing_urls.add(url)

        # Fusionner les nouvelles données avec les données existantes
        all_articles = existing_data + articles_data

        save_to_json(all_articles)

    except Exception as e:
        print(f"Erreur lors de l'exécution du programme : {e}")
    finally:
        driver.quit()

Erreur lors de la vérification périodique des pop-ups : close_all_popups() missing 1 required positional argument: 'driver'
Erreur lors de l'exécution du programme : HTTPConnectionPool(host='localhost', port=50879): Max retries exceeded with url: /session/5a539825e6f74dbd8bd1248564f366d1/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000273EE9DBC20>: Failed to establish a new connection: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée'))


Erreur lors de la recherche de l'élément div[id*="consent"] : HTTPConnectionPool(host='localhost', port=50879): Max retries exceeded with url: /session/5a539825e6f74dbd8bd1248564f366d1/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000273EE9D9310>: Failed to establish a new connection: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée'))
Erreur lors de la recherche de l'élément div[id*="overlay"] : HTTPConnectionPool(host='localhost', port=50879): Max retries exceeded with url: /session/5a539825e6f74dbd8bd1248564f366d1/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000273EF5EC470>: Failed to establish a new connection: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée'))
Erreur lors de la recherche de l'élément div[id*="popup"] : HTTPConnectionPool(host='localhost', port=50879): Max retries exceeded w

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import json
import os
import threading

# Initialisation du driver Selenium
driver = webdriver.Chrome()  # Remplace par le driver de ton choix (Chrome, Firefox, etc.)

def close_all_popups():
    """
    Ferme tous les pop-ups, notifications et publicités détectés.
    """
    popup_selectors = [
        'div[class*="popup"]',  # Pop-ups génériques
        'div[class*="modal"]',  # Modales
        'div[class*="advertisement"]',  # Publicités
        'div[class*="notification"]',  # Notifications
        'button[class*="close"]',  # Boutons de fermeture
        'button[class*="dismiss"]',  # Boutons de rejet
        'button[class*="decline"]',  # Boutons de refus (ex: cookies)
        'img[class*="ad"]',  # Images publicitaires
        'iframe[src*="ads"]',  # Iframes de publicités
        'div[id*="cookie"]',  # Bannières de cookies
        'div[id*="consent"]',  # Bannières de consentement
        'div[id*="overlay"]',  # Overlays
        'div[id*="popup"]',  # Pop-ups par ID
    ]

    for selector in popup_selectors:
        try:
            elements = driver.find_elements(By.CSS_SELECTOR, selector)
            for element in elements:
                try:
                    if "close" in selector.lower():
                        element.click()
                    else:
                        driver.execute_script("arguments[0].style.display = 'none';", element)
                    print(f"Élément fermé ou masqué avec le sélecteur : {selector}")
                except Exception as e:
                    print(f"Erreur lors de la fermeture de l'élément {selector} : {e}")
        except Exception as e:
            print(f"Erreur lors de la recherche de l'élément {selector} : {e}")

def check_for_popups_periodically(interval=2):
    """
    Vérifie périodiquement la présence de pop-ups et les ferme.
    :param interval: Intervalle de vérification en secondes
    """
    try:
        while True:
            close_all_popups()
            time.sleep(interval)
    except Exception as e:
        print(f"Erreur lors de la vérification périodique des pop-ups : {e}")

def click_sur_plus():
    """
    Charge la page et clique sur le bouton "Afficher plus" pour charger plus d'articles.
    """
    driver.get('https://sport.le360.ma/')
    close_all_popups()

    try:
        previous_article_count = 0
        for _ in range(500):
            close_all_popups()
            bouton_afficher_plus = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '.results-list-load-more'))
            )
            bouton_afficher_plus.click()
            time.sleep(2)

            # Vérifier si de nouveaux articles ont été chargés
            current_article_count = len(get_article_links(driver))
            if current_article_count == previous_article_count:
                print("Aucun nouvel article chargé, arrêt du clic sur 'Afficher plus'.")
                break
            previous_article_count = current_article_count

    except Exception as e:
        print(f"Erreur lors du clic sur 'Afficher plus' : {e}")
    finally:
        print("Chargement des articles terminé.")

def get_article_links(driver, max_articles=1500):
    """
    Récupère les liens des articles.
    :param driver: Instance du navigateur Selenium
    :param categorie: Catégorie des articles à filtrer
    :param max_articles: Nombre maximum d'articles à récupérer
    :return: Liste des liens des articles
    """
    for _ in range(2):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = soup.find_all('a', href=True)
    article_links = set()

    for article in articles:
        if len(article_links) >= max_articles:
            break
        link = article['href']
        if link.startswith('/football/'):
            article_links.add(link)

    return list(article_links)

def scrape_article(driver, article_url):
    """
    Scrape les détails d'un article à partir de son URL.
    :param driver: Instance du navigateur Selenium
    :param article_url: URL de l'article à scraper
    :return: Dictionnaire contenant le titre, la date, la catégorie et le contenu de l'article
    """
    try:
        driver.execute_script(f"window.open('{article_url}', '_blank');")
        driver.switch_to.window(driver.window_handles[1])
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        close_all_popups()

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        titre = extract_titre(soup)
        date = extract_date(soup)
        categorie = extract_categorie(soup)
        contenu = extract_contenu(soup)

    except Exception as e:
        print(f"Erreur lors du scraping de l'article {article_url}: {e}")
        titre, date, categorie, contenu = "Titre non trouvé", "Date non trouvée", "Catégorie non trouvée", "Contenu non trouvé"

    finally:
        try:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except Exception as e:
            print(f"Erreur lors de la fermeture de l'onglet : {e}")

    return {
        'titre': titre,
        'date': date,
        'categorie': categorie,
        'contenu': contenu,
    }

def extract_titre(soup):
    """Extrait le titre de l'article."""
    try:
        return soup.find('h1').text.strip()
    except AttributeError:
        return "Titre non trouvé"

def extract_date(soup):
    """Extrait la date de l'article."""
    try:
        return soup.find('div', {"class": "subheadline-date"}).text.strip()
    except AttributeError:
        return "Date non trouvée"

def extract_categorie(soup):
    """Extrait la catégorie de l'article."""
    try:
        return soup.find('a', {"class": "overline-link"}).text.strip()
    except AttributeError:
        return "Catégorie non trouvée"

def extract_contenu(soup):
    """Extrait le contenu de l'article."""
    try:
        paragraphes = soup.find_all('p')
        return " ".join([p.text.strip() for p in paragraphes])
    except AttributeError:
        return "Contenu non trouvé"

def load_existing_data(filename="articles.json"):
    """
    Charge les données existantes depuis un fichier JSON.
    :param filename: Nom du fichier JSON
    :return: Liste des articles existants
    """
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            return json.load(f)
    return []

def save_to_json(data, filename="articles.json"):
    """
    Sauvegarde les données dans un fichier JSON.
    :param data: Données à sauvegarder
    :param filename: Nom du fichier JSON
    """
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"Données sauvegardées dans {filename}")

def load_scraped_urls(filename="le360.json"):
    """
    Charge les URLs déjà scrapés depuis un fichier JSON.
    :param filename: Nom du fichier JSON
    :return: Un ensemble des URLs déjà scrapés
    """
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            data = json.load(f)
            return set(article['url'] for article in data)
    return set()

if __name__ == "__main__":
    try:
        popup_thread = threading.Thread(target=check_for_popups_periodically, daemon=True)
        popup_thread.start()

        click_sur_plus()

        article_urls = get_article_links(driver, max_articles=1200)
        print(f"{len(article_urls)} articles trouvés.")

        scraped_urls = load_scraped_urls()

        existing_data = load_existing_data()
        existing_urls = set(article['url'] for article in existing_data)

        articles_data = []
        for url in article_urls:
            if url in existing_urls:
                print(f"Article déjà présent : {url}")
                continue

            print(f"Scraping de l'article : {url}")
            article_data = scrape_article(driver, url)
            article_data['url'] = url
            articles_data.append(article_data)
            existing_urls.add(url)

        # Fusionner les nouvelles données avec les données existantes
        all_articles = existing_data + articles_data

        save_to_json(all_articles)

    except Exception as e:
        print(f"Erreur lors de l'exécution du programme : {e}")
    finally:
        driver.quit()

Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Erreur lors du clic sur 'Afficher plus' : Message: element click intercepted: Element <button class="results-list-load-more">...</button> is not clickable at point (387, 19). Other element would receive the click: <div class="header-360_nav-wrapper header-360_nav-wrapper-fixed">...</div>
  (Session info: chrome=133.0.6943.127)
Stacktrace:
	GetHandleVerifier [0x00007FF683616EE5+28773]
	(No symbol) [0x00007FF6835825D0]
	(No symbol) [0x00007FF683418FAA]
	(No symbol) [0x00007FF683477109]
	(No symbol) [0x00007FF683474AD2]
	(No symbol) [0x00007FF683471B81]
	(No symbol) [0x

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import json
import os
import threading

# Initialisation du driver Selenium
driver = webdriver.Chrome()  # Remplace par le driver de ton choix (Chrome, Firefox, etc.)

def close_all_popups():
    """
    Ferme tous les pop-ups, notifications et publicités détectés.
    """
    popup_selectors = [
        'div[class*="popup"]',  # Pop-ups génériques
        'div[class*="modal"]',  # Modales
        'div[class*="advertisement"]',  # Publicités
        'div[class*="notification"]',  # Notifications
        'button[class*="close"]',  # Boutons de fermeture
        'button[class*="dismiss"]',  # Boutons de rejet
        'button[class*="decline"]',  # Boutons de refus (ex: cookies)
        'img[class*="ad"]',  # Images publicitaires
        'iframe[src*="ads"]',  # Iframes de publicités
        'div[id*="cookie"]',  # Bannières de cookies
        'div[id*="consent"]',  # Bannières de consentement
        'div[id*="overlay"]',  # Overlays
        'div[id*="popup"]',  # Pop-ups par ID
    ]

    for selector in popup_selectors:
        try:
            elements = driver.find_elements(By.CSS_SELECTOR, selector)
            for element in elements:
                try:
                    if "close" in selector.lower():
                        element.click()
                    else:
                        driver.execute_script("arguments[0].style.display = 'none';", element)
                    print(f"Élément fermé ou masqué avec le sélecteur : {selector}")
                except Exception as e:
                    print(f"Erreur lors de la fermeture de l'élément {selector} : {e}")
        except Exception as e:
            print(f"Erreur lors de la recherche de l'élément {selector} : {e}")

def check_for_popups_periodically(interval=2):
    """
    Vérifie périodiquement la présence de pop-ups et les ferme.
    :param interval: Intervalle de vérification en secondes
    """
    try:
        while True:
            close_all_popups()
            time.sleep(interval)
    except Exception as e:
        print(f"Erreur lors de la vérification périodique des pop-ups : {e}")

def click_sur_plus():
    """
    Charge la page et clique sur le bouton "Afficher plus" pour charger plus d'articles.
    """
    driver.get('https://fr.le360.ma/lifestyle/')
    close_all_popups()

    try:
        previous_article_count = 0
        for _ in range(500):
            close_all_popups()
            bouton_afficher_plus = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.results-list-load-more'))
            )
            # Cliquer sur le bouton via JavaScript pour éviter les interceptions
            driver.execute_script("arguments[0].click();", bouton_afficher_plus)
            time.sleep(2)

            # Vérifier si de nouveaux articles ont été chargés
            current_article_count = len(get_article_links(driver))
            if current_article_count == previous_article_count:
                print("Aucun nouvel article chargé, arrêt du clic sur 'Afficher plus'.")
                break
            previous_article_count = current_article_count

    except Exception as e:
        print(f"Erreur lors du clic sur 'Afficher plus' : {e}")
    finally:
        print("Chargement des articles terminé.")

def get_article_links(driver, max_articles=1500):
    """
    Récupère les liens des articles.
    :param driver: Instance du navigateur Selenium
    :param max_articles: Nombre maximum d'articles à récupérer
    :return: Liste des liens des articles
    """
    for _ in range(2):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = soup.find_all('a', href=True)
    article_links = set()

    for article in articles:
        if len(article_links) >= max_articles:
            break
        link = article['href']
        if link.startswith('/lifestyle/'):
            article_links.add(link)

    return list(article_links)

def scrape_article(driver, article_url):
    """
    Scrape les détails d'un article à partir de son URL.
    :param driver: Instance du navigateur Selenium
    :param article_url: URL de l'article à scraper
    :return: Dictionnaire contenant le titre, la date, la catégorie et le contenu de l'article
    """
    try:
        driver.execute_script(f"window.open('{article_url}', '_blank');")
        driver.switch_to.window(driver.window_handles[1])
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        close_all_popups()

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        titre = extract_titre(soup)
        date = extract_date(soup)
        categorie = extract_categorie(soup)
        contenu = extract_contenu(soup)

    except Exception as e:
        print(f"Erreur lors du scraping de l'article {article_url}: {e}")
        titre, date, categorie, contenu = "Titre non trouvé", "Date non trouvée", "Catégorie non trouvée", "Contenu non trouvé"

    finally:
        try:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except Exception as e:
            print(f"Erreur lors de la fermeture de l'onglet : {e}")

    return {
        'titre': titre,
        'date': date,
        'categorie': categorie,
        'contenu': contenu,
    }

def extract_titre(soup):
    """Extrait le titre de l'article."""
    try:
        return soup.find('h1').text.strip()
    except AttributeError:
        return "Titre non trouvé"

def extract_date(soup):
    """Extrait la date de l'article."""
    try:
        return soup.find('div', {"class": "subheadline-date"}).text.strip()
    except AttributeError:
        return "Date non trouvée"

def extract_categorie(soup):
    """Extrait la catégorie de l'article."""
    try:
        return soup.find('a', {"class": "overline-link"}).text.strip()
    except AttributeError:
        return "Catégorie non trouvée"

def extract_contenu(soup):
    """Extrait le contenu de l'article."""
    try:
        paragraphes = soup.find_all('p')
        return " ".join([p.text.strip() for p in paragraphes])
    except AttributeError:
        return "Contenu non trouvé"

def load_existing_data(filename="articles.json"):
    """
    Charge les données existantes depuis un fichier JSON.
    :param filename: Nom du fichier JSON
    :return: Liste des articles existants
    """
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            return json.load(f)
    return []

def save_to_json(data, filename="articles.json"):
    """
    Sauvegarde les données dans un fichier JSON.
    :param data: Données à sauvegarder
    :param filename: Nom du fichier JSON
    """
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"Données sauvegardées dans {filename}")

def load_scraped_urls(filename="le360.json"):
    """
    Charge les URLs déjà scrapés depuis un fichier JSON.
    :param filename: Nom du fichier JSON
    :return: Un ensemble des URLs déjà scrapés
    """
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            data = json.load(f)
            return set(article['url'] for article in data)
    return set()

if __name__ == "__main__":
    try:
        popup_thread = threading.Thread(target=check_for_popups_periodically, daemon=True)
        popup_thread.start()

        click_sur_plus()

        article_urls = get_article_links(driver, max_articles=1200)
        print(f"{len(article_urls)} articles trouvés.")

        scraped_urls = load_scraped_urls()

        existing_data = load_existing_data()
        existing_urls = set(article['url'] for article in existing_data)

        articles_data = []
        for url in article_urls:
            if url in existing_urls:
                print(f"Article déjà présent : {url}")
                continue

            print(f"Scraping de l'article : {url}")
            article_data = scrape_article(driver, url)
            article_data['url'] = url
            articles_data.append(article_data)
            existing_urls.add(url)

        # Fusionner les nouvelles données avec les données existantes
        all_articles = existing_data + articles_data

        save_to_json(all_articles)

    except Exception as e:
        print(f"Erreur lors de l'exécution du programme : {e}")
    finally:
        driver.quit()

Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué 

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import json
import os
import threading

# Initialisation du driver Selenium pour Firefox
driver = webdriver.Firefox()  # Utilisation de Firefox

def close_all_popups():
    """
    Ferme tous les pop-ups, notifications et publicités détectés.
    """
    popup_selectors = [
        'div[class*="popup"]',  # Pop-ups génériques
        'div[class*="modal"]',  # Modales
        'div[class*="advertisement"]',  # Publicités
        'div[class*="notification"]',  # Notifications
        'button[class*="close"]',  # Boutons de fermeture
        'button[class*="dismiss"]',  # Boutons de rejet
        'button[class*="decline"]',  # Boutons de refus (ex: cookies)
        'img[class*="ad"]',  # Images publicitaires
        'iframe[src*="ads"]',  # Iframes de publicités
        'div[id*="cookie"]',  # Bannières de cookies
        'div[id*="consent"]',  # Bannières de consentement
        'div[id*="overlay"]',  # Overlays
        'div[id*="popup"]',  # Pop-ups par ID
    ]

    for selector in popup_selectors:
        try:
            elements = driver.find_elements(By.CSS_SELECTOR, selector)
            for element in elements:
                try:
                    if "close" in selector.lower():
                        element.click()
                    else:
                        driver.execute_script("arguments[0].style.display = 'none';", element)
                    print(f"Élément fermé ou masqué avec le sélecteur : {selector}")
                except Exception as e:
                    print(f"Erreur lors de la fermeture de l'élément {selector} : {e}")
        except Exception as e:
            print(f"Erreur lors de la recherche de l'élément {selector} : {e}")

def check_for_popups_periodically(interval=2):
    """
    Vérifie périodiquement la présence de pop-ups et les ferme.
    :param interval: Intervalle de vérification en secondes
    """
    try:
        while True:
            close_all_popups()
            time.sleep(interval)
    except Exception as e:
        print(f"Erreur lors de la vérification périodique des pop-ups : {e}")

def click_sur_plus():
    """
    Charge la page et clique sur le bouton "Afficher plus" pour charger plus d'articles.
    """
    driver.get('https://fr.le360.ma/people/')
    close_all_popups()

    try:
        previous_article_count = 0
        for _ in range(500):
            close_all_popups()
            bouton_afficher_plus = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.results-list-load-more'))
            )
            # Cliquer sur le bouton via JavaScript pour éviter les interceptions
            driver.execute_script("arguments[0].click();", bouton_afficher_plus)
            time.sleep(2)

            # Vérifier si de nouveaux articles ont été chargés
            current_article_count = len(get_article_links(driver))
            if current_article_count == previous_article_count:
                print("Aucun nouvel article chargé, arrêt du clic sur 'Afficher plus'.")
                break
            previous_article_count = current_article_count

    except Exception as e:
        print(f"Erreur lors du clic sur 'Afficher plus' : {e}")
    finally:
        print("Chargement des articles terminé.")

def get_article_links(driver, max_articles=1500):
    """
    Récupère les liens des articles.
    :param driver: Instance du navigateur Selenium
    :param max_articles: Nombre maximum d'articles à récupérer
    :return: Liste des liens des articles
    """
    for _ in range(2):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = soup.find_all('a', href=True)
    article_links = set()

    for article in articles:
        if len(article_links) >= max_articles:
            break
        link = article['href']
        if link.startswith('/people/'):
            article_links.add(link)

    return list(article_links)

def scrape_article(driver, article_url):
    """
    Scrape les détails d'un article à partir de son URL.
    :param driver: Instance du navigateur Selenium
    :param article_url: URL de l'article à scraper
    :return: Dictionnaire contenant le titre, la date, la catégorie et le contenu de l'article
    """
    try:
        driver.execute_script(f"window.open('{article_url}', '_blank');")
        driver.switch_to.window(driver.window_handles[1])
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        close_all_popups()

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        titre = extract_titre(soup)
        date = extract_date(soup)
        categorie = extract_categorie(soup)
        contenu = extract_contenu(soup)

    except Exception as e:
        print(f"Erreur lors du scraping de l'article {article_url}: {e}")
        titre, date, categorie, contenu = "Titre non trouvé", "Date non trouvée", "Catégorie non trouvée", "Contenu non trouvé"

    finally:
        try:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except Exception as e:
            print(f"Erreur lors de la fermeture de l'onglet : {e}")

    return {
        'titre': titre,
        'date': date,
        'categorie': categorie,
        'contenu': contenu,
    }

def extract_titre(soup):
    """Extrait le titre de l'article."""
    try:
        return soup.find('h1').text.strip()
    except AttributeError:
        return "Titre non trouvé"

def extract_date(soup):
    """Extrait la date de l'article."""
    try:
        return soup.find('div', {"class": "subheadline-date"}).text.strip()
    except AttributeError:
        return "Date non trouvée"

def extract_categorie(soup):
    """Extrait la catégorie de l'article."""
    try:
        return soup.find('a', {"class": "overline-link"}).text.strip()
    except AttributeError:
        return "Catégorie non trouvée"

def extract_contenu(soup):
    """Extrait le contenu de l'article."""
    try:
        paragraphes = soup.find_all('p')
        return " ".join([p.text.strip() for p in paragraphes])
    except AttributeError:
        return "Contenu non trouvé"

def load_existing_data(filename="articles.json"):
    """
    Charge les données existantes depuis un fichier JSON.
    :param filename: Nom du fichier JSON
    :return: Liste des articles existants
    """
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            return json.load(f)
    return []

def save_to_json(data, filename="articles.json"):
    """
    Sauvegarde les données dans un fichier JSON.
    :param data: Données à sauvegarder
    :param filename: Nom du fichier JSON
    """
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"Données sauvegardées dans {filename}")

def load_scraped_urls(filename="le360.json"):
    """
    Charge les URLs déjà scrapés depuis un fichier JSON.
    :param filename: Nom du fichier JSON
    :return: Un ensemble des URLs déjà scrapés
    """
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            data = json.load(f)
            return set(article['url'] for article in data)
    return set()

if __name__ == "__main__":
    try:
        popup_thread = threading.Thread(target=check_for_popups_periodically, daemon=True)
        popup_thread.start()

        click_sur_plus()

        article_urls = get_article_links(driver, max_articles=1200)
        print(f"{len(article_urls)} articles trouvés.")

        scraped_urls = load_scraped_urls()

        existing_data = load_existing_data()
        existing_urls = set(article['url'] for article in existing_data)

        articles_data = []
        for url in article_urls:
            if url in existing_urls:
                print(f"Article déjà présent : {url}")
                continue

            print(f"Scraping de l'article : {url}")
            article_data = scrape_article(driver, url)
            article_data['url'] = url
            articles_data.append(article_data)
            existing_urls.add(url)

        # Fusionner les nouvelles données avec les données existantes
        all_articles = existing_data + articles_data

        save_to_json(all_articles)

    except Exception as e:
        print(f"Erreur lors de l'exécution du programme : {e}")
    finally:
        driver.quit()

Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué avec le sélecteur : img[class*="ad"]
Élément fermé ou masqué 